In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import plotly.express as px

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Importation du jeu de données

In [ ]:
df_2018 = pd.read_csv('valeursfoncieres-2018.txt', sep="|", decimal=",")
df_2019 = pd.read_csv('valeursfoncieres-2019.txt', sep="|", decimal=",")
df_2020 = pd.read_csv('valeursfoncieres-2020.txt', sep="|", decimal=",")
df_2021 = pd.read_csv('valeursfoncieres-2021.txt', sep="|", decimal=",")

In [ ]:
df_2018.shape[0] + df_2019.shape[0] + df_2020.shape[0] + df_2021.shape[0]

In [ ]:
df = pd.concat([df_2018,df_2019, df_2020, df_2021], axis=0)

In [ ]:
df.shape

In [ ]:
# On paramétrise notre display function pour avoir toutes les variables
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df.loc[df["Code departement"] == 67]

In [ ]:
df.loc[df["Code departement"] == 68]

In [ ]:
df.loc[df["Code departement"] == 57]

In [ ]:
# On extrait les noms des variables
df.columns

In [ ]:
# On regarde le nombre de valeurs unique dans chaque colonne pour enlever les variables qui n'apporte pas d'information
for col in df.columns:
    print(col)
    print(len(df[col].unique()))

# Nettoyage du jeu de données

In [ ]:
# On enlève donc les variables avec une valeur (souvent NaN) ou d'identifiant (qui n'auront donc pas d'intérêt dans la prédiction ou l'exploration)
df.drop(columns=['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI','No disposition', 'No plan', 'Identifiant local', 'No Volume'], inplace=True)

In [ ]:
df.head()

B/T/Q représente l'indice de répétition qui est est une mention qui complète une numérotation de voirie. L’indice de répétition permet de
différencier plusieurs adresses portant le même numéro dans la même rue. Elle ne nous servira pas en soit car donc pas car 

In [ ]:
df.drop(columns=["B/T/Q"], inplace=True)

In [ ]:
df.head(n= 10)

In [ ]:
# On regarde le nombre de valeurs manquantes dans chaque colonne pour enlever les variables qui n'apporte pas d'information
for col in df.columns:
    print(col)
    print(df[col].isnull().sum()/df.shape[0])

Le nombre de lot est suffisant pour nous indiquer la présence ou non de lots. Le reste est redondant.

In [ ]:
df.drop(columns=['1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot'], inplace=True)

In [ ]:
df.head(n=10)

Nous nous occupons désormais de la variable nature culture spéciale qui nous indique la présence ou non d'un extérieur

In [ ]:
df.head()

In [ ]:
df.drop(columns=[ "Nature culture speciale"], inplace=True)

In [ ]:
df.head(n=10)

In [ ]:
df.loc[df["Nature culture"].isna(), "exterieur"] = 0
df.loc[df["exterieur"].isna(), "exterieur"] = 1

df.head()

In [ ]:
# La présence d'un terrain est clairement indiquer par la surface présente, donc si elle est de 0: il n'y a pas de terrain
df.loc[df["Surface terrain"].isnull(), "Surface terrain"] = 0

In [ ]:
df.head(n=10)

In [ ]:
# En s'appuyant sur la documentation, on se rend compte que le relatif aux sections n'est pas pertinent
df.drop(columns=["Prefixe de section", "Section"], inplace=True)

In [ ]:
df.head()

In [ ]:
# On extrait les différentes modalités prises par Nature mutation
df["Nature mutation"].unique()

In [ ]:
# On observe que ce n'est pas pertinent pour notre problématique.
df.drop(columns=["Nature mutation"], inplace=True)

In [ ]:
df.head()

La variable en relation avec l'adresse se sépare en plusieurs variables, nous allons donc la concaténer. Si l'adresse est nulle, c'est parce qu'il n'y a pas de n° de voie (lieux dits?), donc on remplie adresse avec "Voie".

In [ ]:
df["key"]=  df["No voie"].astype(str)+df["Voie"]+df["Commune"]+df["Date mutation"]
df.head(n=10)

In [ ]:
df.head()

In [ ]:
# On enlève les références à l'adresse qui ne nous seront d'aucunes utilité
df.drop(columns=["Type de voie", "Voie", "No voie", "Code voie"], inplace=True)

In [ ]:
df.head(n=10)

In [ ]:
# On enlève également le code département qui ne nous servira pas pour récupérer les départements (nous utiliserons le code postal et le nom de la commune)
df.drop(columns=["Code departement"], inplace=True)

In [ ]:
df.head()

In [ ]:
# Pour ne garder les ventes que de lots uniques, on enlève toutes les ventes qui se répètent plus d'une fois
df2 = df.drop_duplicates(subset='key', keep=False)

In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
# On change le format de la date de vente en format datetime.
df2.loc[:, 'Date mutation'] = pd.to_datetime(df2['Date mutation'] , format='%d/%m/%Y')

In [ ]:
# On récupère le mois et l'année de la vente.
df2.loc[:, 'year'] = pd.DatetimeIndex(df2['Date mutation']).year
df2.loc[:, 'month'] = pd.DatetimeIndex(df2['Date mutation']).month

In [ ]:
df2.head()

In [ ]:
# On enlève la clé qui nous a servi à enlever les duplicatas et la date entière qui ne nous sert plus
df2.drop(columns=["Date mutation", "key"], inplace=True)

In [ ]:
df2.head()

In [ ]:
# Si NaN values dans ces variables de surfaces ou nombre de pièces principales, 0 assigné
df2.loc[df2["Surface reelle bati"].isnull(), "Surface reelle bati"] = 0
df2.loc[df2["Surface terrain"].isnull(), "Surface terrain"] = 0
df2.loc[df2["Nombre pieces principales"].isnull(), "Nombre pieces principales"] = 0

In [ ]:
# On enlève la clé qui nous a servi à enlever les duplicatas et la date entière qui ne nous sert plus
df2.drop(columns=["Nature culture"], inplace=True)

In [ ]:
# On enlève la clé qui nous a servi à enlever les duplicatas et la date entière qui ne nous sert plus
df2.drop(columns=["Code type local"], inplace=True)

In [ ]:
df2.head()

In [ ]:
df2.loc[df2["Type local"].isna(), "Type local"] = "Autres"

In [ ]:
df2.head()

In [ ]:
for col in df2.columns:
    print(col)
    print(df2[col].isnull().sum())

In [ ]:
# On enlève les lignes avec des valeurs nulles
df2 = df2.dropna()

In [ ]:
for col in df2.columns:
    print(col)
    print(df2[col].isnull().sum())

Certains biens sont à l'euro symbolique ou ont un prix inférieur à 100€, nous enlevons ces biens

In [ ]:
# On visualise le pourcentage de biens avec un prix de vente inférieur à 100€.
euro_symbol = (df2["Valeur fonciere"] < 100).value_counts()

plt.pie(euro_symbol, autopct='%.0f%%')

In [ ]:
# On ne garde que les biens ayant eu un prix de vente supérieur à 100€.
df3 = df2[df2["Valeur fonciere"] > 100]
df3.head()

In [ ]:
# On récupère les différents quartiles pour pouvoir enlever les biens avec un prix anormalement hauts comparés aux autres biens.
Q1, Q2, Q3 = df3['Valeur fonciere'].quantile([0.25, 0.5, 0.75])

# On calcule le rang interquartile
IQR = Q3-Q1

In [ ]:
# On détexte les biens ayant un prix supérieur au troisième quartile multiplié par 1.5 fois le rang interquartile
outliers_max = df3["Valeur fonciere"] > (Q3 + 1.5*(IQR))
plt.pie(outliers_max.value_counts(), autopct='%.0f%%')

In [ ]:
df3.shape

In [ ]:
# On ne garde que les biens n'ayant pas un prix de vente anormalement haut.
df3 = df3[df3["Valeur fonciere"] < (Q3 + 1.5*(IQR))]

# Liaison avec données des départements et communes

In [ ]:
# On récupère la base de données des communes/département
departement = pd.read_csv("communes-departement-region.csv")
departement.head()

In [ ]:
# On ne garde que les variables qui nous serviront à cartographier nos données
departement  = departement.loc[:,["code_postal", "nom_commune", "nom_departement","nom_region", "latitude", "longitude"]]
# On met en majuscule le nom des communes pour joindre les bases de données
departement["nom_commune"] = departement["nom_commune"].str.upper()
departement.head()

In [ ]:
df3.shape

In [ ]:
# On fait une jointure interne entre les deux bases de données
df4  = pd.merge(df3, departement, left_on=['Commune', 'Code postal'], right_on=['nom_commune', 'code_postal'], how='inner')
df4.head()

In [ ]:
df4.shape

In [ ]:
# On change le type de pièces principales pour la visualisation
df4["Nombre pieces principales"] = df4["Nombre pieces principales"].astype(int)

In [ ]:
# On raccourcit le nom de ce type de local pour la visualisation
df4.loc[df4["Type local"] == 'Local industriel. commercial ou assimilé', "Type local"] = "Local"

In [ ]:
fig, axes = plt.subplots(2, 3,figsize=(30,12))

#create boxplot in each subplot
sns.boxplot(data=df4, x="month", y="Valeur fonciere", ax=axes[0,0])
sns.boxplot(data=df4, x="Type local", y="Valeur fonciere", ax=axes[0,1])
sns.boxplot(data=df4, x="Nombre de lots", y="Valeur fonciere", ax=axes[0,2])
sns.boxplot(data=df4, x="Nombre pieces principales", y="Valeur fonciere", ax=axes[1,0])
sns.scatterplot(data=df4, x="Surface reelle bati", y="Valeur fonciere", ax=axes[1,1])
sns.scatterplot(data=df4, x="Surface terrain", y="Valeur fonciere", ax=axes[1,2])

In [ ]:
month_dict_fr = {
    1: 'Janvier',
    2: 'Février',
    3: 'Mars',
    4: 'Avril',
    5: 'Mai',
    6: 'Juin',
    7: 'Juillet',
    8: 'Août',
    9: 'Septembre',
    10: 'Octobre',
    11: 'Novembre',
    12: 'Décembre'
}


In [ ]:
df4["month"] = [month_dict_fr[month] for month in df4["month"]]

In [ ]:
order_month = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 'Juillet', 'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre']


In [ ]:
fig = px.box(df4, x="month", y="Valeur fonciere", category_orders={'month': order_month}, color="month")

fig.show()

In [ ]:
pivot_departement = df4.pivot_table(index=["nom_departement","month"], values="Valeur fonciere", aggfunc="mean").reset_index()
pivot_departement

In [ ]:
months_order = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 'Juillet', 'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre']

# Create a categorical data type with the order of months
month_cat = pd.CategoricalDtype(categories=months_order, ordered=True)

# Convert the 'month' column to the categorical data type
pivot_departement['month'] = pivot_departement['month'].astype(month_cat)

# Now sort by the 'month' column
pivot_departement = pivot_departement.sort_values(by='month')

In [ ]:

fig = px.line(pivot_departement, x="month", y="Valeur fonciere", color='nom_departement')
fig.show()

In [ ]:
pivot_region = df4.pivot_table(index=["nom_region","month"], values="Valeur fonciere", aggfunc="mean").reset_index()

# Convert the 'month' column to the categorical data type
pivot_region['month'] = pivot_region['month'].astype(month_cat)

# Now sort by the 'month' column
pivot_region = pivot_region.sort_values(by='month')

In [ ]:

fig = px.line(pivot_region, x="month", y="Valeur fonciere", color='nom_region')
fig.show()

In [ ]:
fig = plt.subplots(1, 1,figsize=(30,12))
sns.boxplot(data=df4, x="nom_departement", y="Valeur fonciere")
plt.xticks(rotation=45, ha='right')


In [ ]:
for region in df4["nom_region"].unique():
    print(region)

In [ ]:
# Pour chaque region, on construit le boxplot des départements correspondant: 
for region in df4["nom_region"].unique():
    df_graph = df4.loc[df4["nom_region"] == region]
    fig = px.box(df_graph, x="nom_departement", y="Valeur fonciere",  color="nom_departement",  title=region)
    fig.update_layout(showlegend=False)
    fig.show()

In [ ]:
fig = plt.subplots(1, 1,figsize=(20,12))
sns.boxplot(data=df4, x="nom_region", y="Valeur fonciere")
plt.xticks(rotation=45, ha='right')

In [ ]:
fig = px.box(df4, x="nom_region", y="Valeur fonciere",  color="nom_region")

fig.show()

# Map

In [ ]:
df4.head()

In [ ]:
mean_values = pd.pivot_table(data=df4, index=["longitude", "latitude"], values="Valeur fonciere", aggfunc="mean")
mean_values = mean_values.reset_index()
mean_values

In [ ]:
mean_values.head()

In [ ]:
mean_values['normalized_value'] = (mean_values['Valeur fonciere'] -mean_values['Valeur fonciere'].min()) / (mean_values['Valeur fonciere'].max() - mean_values['Valeur fonciere'].min())
mean_values['normalized_value'].plot.hist(bins=30, alpha=0.5)


In [ ]:
fig = px.scatter_geo(mean_values,
                     lat='latitude',
                     lon='longitude',
                     scope='europe', 
                    color='Valeur fonciere',
                     color_continuous_scale='Inferno',
                     # Centré sur la france
                     center=dict(lat=46.26, lon=2.52),
                     # Normalisé car doit être en 0 et 1
                     opacity=mean_values["normalized_value"])

#On update la map pour la centrer sur la france
fig.update_layout(
    autosize=True,
    height=600,
    geo=dict(
        center=dict(
            lat=46.26,
            lon=2.52
        ),
        scope='europe',
        projection_scale=6
    )
)
fig.show()

### Polygons de DataGouv: https://www.data.gouv.fr/fr/datasets/contours-des-communes-de-france-simplifie-avec-regions-et-departement-doutre-mer-rapproches/

In [ ]:
import json

In [ ]:
mean_values_json = pd.pivot_table(data=df4, index=["nom_region"], values="Valeur fonciere", aggfunc="mean")
mean_values_json = mean_values_json.reset_index()
mean_values_json

In [ ]:
mean_values_json.loc[mean_values_json["nom_region"] == "Nouvelle-Aquitaine", "nom_region"] = "Nouvelle Aquitaine"
mean_values_json.loc[mean_values_json["nom_region"] == "Grand Est", "nom_region"] = "Grand-Est"

In [ ]:
with open("regions.json", "r") as file:
    regions_data = json.load(file)

In [ ]:
# Create the choropleth map
fig = px.choropleth(
    mean_values_json,  # replace df with your DataFrame
    geojson=regions_data,
    locations='nom_region',  # replace 'id' with the column name containing the regions' ids
    color='Valeur fonciere',  # replace 'value' with the column name containing the values you want to plot
    color_continuous_scale='Inferno',
    featureidkey="properties.libgeo",  # replace 'properties.id' with the path to the ids in the geojson
    range_color=[100000, 250000]
)

# Update the geos layout to focus on France
fig.update_geos(
    center={"lat": 46.603354, "lon": 1.888334},  # Coordinates of France's centroid
    projection_scale=15,  # Adjust the scale to fit France
    visible=False  # Hide the base map
)

# Update the layout
fig.update_layout(
    title="Choropleth Map of France",
    margin={"r":0,"t":40,"l":0,"b":0}
)

# Show the map
fig.show()

In [ ]:
with open("departement.json", "r") as file:
    dep_data = json.load(file)

In [ ]:
mean_values_json_departement = pd.pivot_table(data=df4, index=["nom_departement"], values="Valeur fonciere", aggfunc="mean")
mean_values_json_departement = mean_values_json_departement.reset_index()
mean_values_json_departement

In [ ]:
# Create the choropleth map
fig = px.choropleth(
    mean_values_json_departement,  # replace df with your DataFrame
    geojson=dep_data,
    locations='nom_departement',  # replace 'id' with the column name containing the regions' ids
    color='Valeur fonciere',  # replace 'value' with the column name containing the values you want to plot
    color_continuous_scale='Inferno',
    featureidkey="properties.libgeo"  # replace 'properties.id' with the path to the ids in the geojson
)

# Update the geos layout to focus on France
fig.update_geos(
    center={"lat": 46.603354, "lon": 1.888334},  # Coordinates of France's centroid
    projection_scale=15,  # Adjust the scale to fit France
    visible=False  # Hide the base map
)

# Update the layout
fig.update_layout(
    title="Choropleth Map of France",
    margin={"r":0,"t":40,"l":0,"b":0}
)

# Show the map
fig.show()

# Variables supplémentaires

### Population active par département: https://www.insee.fr/fr/statistiques/2012710#titre-bloc-1

In [ ]:
df4.head()

In [ ]:
pop = pd.read_excel("pop_active.xlsx")
pop.head()

In [ ]:
df4.shape

In [ ]:
# On fait une jointure interne entre les deux bases de données
df5  = pd.merge(df4, pop, on="nom_departement", how='inner')
df5.head()

### Salaire Net Horaire Moyen par département (2021)- https://www.insee.fr/fr/statistiques/2021266

In [ ]:
# On importe le salaire horaire moyen
salaire = pd.read_excel("base-cc-bases-tous-salaries-2021.xlsx")
salaire.head()

In [ ]:
# On fait une jointure interne entre les deux bases de données
df5  = pd.merge(df5, salaire, on="nom_departement", how='inner')
df5.head()

In [ ]:
# On crée une matrice de corrélation
matrix = df5.loc[:,['Valeur fonciere', 'Nombre de lots', 'Surface reelle bati', 'Nombre pieces principales', 'Surface terrain', "pop_active", "salaire_moyen"]].corr()
matrix

# Nombre d'écoles par département
https://www.observatoire-des-territoires.gouv.fr/nombre-decoles-elementaires

In [ ]:
# On importe le dataset avec le nombre d'écoles élémentaires par commune
ecole = pd.read_excel("ecoles2.xlsx")
ecole.head()

In [ ]:
# On fait une jointure interne entre les deux bases de données
df6  = pd.merge(df5, ecole, on="nom_departement", how='inner')
df6.head()

# Prix m² et nbre ventes

https://www.data.gouv.fr/fr/datasets/indicateurs-immobiliers-par-commune-et-par-annee-prix-et-volumes-sur-la-periode-2014-2021/

In [ ]:
indic_immob = pd.read_csv("2021.csv")
indic_immob.head()

,Unnamed: 0,INSEE_COM,Annee,Nb_mutations,NbMaisons,NbApparts,PrixMoyen,Prixm2Moyen,SurfaceMoy
0,1,01001,2021,11,11.0,0.0,274126.818182,2551.272727,107.000000
1,2,01002,2021,3,3.0,0.0,163766.666667,1290.000000,120.666667
2,3,01004,2021,249,147.0,102.0,195581.831325,2313.823293,86.389558
3,4,01005,2021,22,18.0,4.0,290837.590909,2970.045455,99.590909
4,5,01006,2021,5,4.0,1.0,171560.000000,1631.800000,104.800000


In [ ]:
indic_immob.loc[indic_immob["INSEE_COM"] == "78683"]

,Unnamed: 0,INSEE_COM,Annee,Nb_mutations,NbMaisons,NbApparts,PrixMoyen,Prixm2Moyen,SurfaceMoy
27030,27031,78683,2021,46,44.0,2.0,460481.1,3932.761,126.1087


In [ ]:
indic_immob.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30876 entries, 0 to 30875
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    30876 non-null  int64  
 1   INSEE_COM     30875 non-null  object 
 2   Annee         30876 non-null  int64  
 3   Nb_mutations  30876 non-null  int64  
 4   NbMaisons     30875 non-null  float64
 5   NbApparts     30875 non-null  float64
 6   PrixMoyen     30876 non-null  float64
 7   Prixm2Moyen   30876 non-null  float64
 8   SurfaceMoy    30876 non-null  float64
dtypes: float64(5), int64(3), object(1)
memory usage: 2.1+ MB


In [ ]:
communes = pd.read_csv("communes-departement-region.csv")
communes.head()

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.0,Auvergne-Rhône-Alpes
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.0,Auvergne-Rhône-Alpes
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.0,Auvergne-Rhône-Alpes
4,1006,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes


In [ ]:
communes.loc[communes["code_commune_INSEE"] == "78683"]

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
34225,78683,VILLIERS ST FREDERIC,78640,VILLIERS ST FREDERIC,NaN,48.820178,1.882508,683.0,NaN,Villiers-Saint-Frédéric,Villiers-Saint-Frédéric,78,Yvelines,11.0,Île-de-France


In [ ]:
communes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39201 entries, 0 to 39200
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   code_commune_INSEE    39201 non-null  object 
 1   nom_commune_postal    39201 non-null  object 
 2   code_postal           39201 non-null  int64  
 3   libelle_acheminement  39201 non-null  object 
 4   ligne_5               3257 non-null   object 
 5   latitude              38932 non-null  float64
 6   longitude             38932 non-null  float64
 7   code_commune          39194 non-null  float64
 8   article               2580 non-null   object 
 9   nom_commune           39201 non-null  object 
 10  nom_commune_complet   39201 non-null  object 
 11  code_departement      39194 non-null  object 
 12  nom_departement       38934 non-null  object 
 13  code_region           38934 non-null  float64
 14  nom_region            38934 non-null  object 
dtypes: float64(4), int6

In [ ]:
def custom_code(row):
    if len(row) == 4:
        code = "0" + row
        return code
    else:
        return row

communes['INSEE_COM'] = communes["code_commune_INSEE"].apply(custom_code)


In [ ]:
communes.head()

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region,INSEE_COM
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.0,Auvergne-Rhône-Alpes,01001
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.0,Auvergne-Rhône-Alpes,01002
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes,01004
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.0,Auvergne-Rhône-Alpes,01005
4,1006,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes,01006


In [ ]:
# On fait une jointure interne entre les deux bases de données
prix  = pd.merge(communes, indic_immob, on="INSEE_COM", how='inner')
prix.head()

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,...,nom_region,INSEE_COM,Unnamed: 0,Annee,Nb_mutations,NbMaisons,NbApparts,PrixMoyen,Prixm2Moyen,SurfaceMoy
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,...,Auvergne-Rhône-Alpes,01001,1,2021,11,11.0,0.0,274126.818182,2551.272727,107.000000
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,...,Auvergne-Rhône-Alpes,01002,2,2021,3,3.0,0.0,163766.666667,1290.000000,120.666667
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,...,Auvergne-Rhône-Alpes,01004,3,2021,249,147.0,102.0,195581.831325,2313.823293,86.389558
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,...,Auvergne-Rhône-Alpes,01005,4,2021,22,18.0,4.0,290837.590909,2970.045455,99.590909
4,1006,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,...,Auvergne-Rhône-Alpes,01006,5,2021,5,4.0,1.0,171560.000000,1631.800000,104.800000


In [ ]:
prix.loc[prix["nom_departement"] == "Yvelines"]

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,...,nom_region,INSEE_COM,Unnamed: 0,Annee,Nb_mutations,NbMaisons,NbApparts,PrixMoyen,Prixm2Moyen,SurfaceMoy
29104,78003,ABLIS,78660,ABLIS,NaN,48.528326,1.850965,3.0,NaN,Ablis,...,Île-de-France,78003,26778,2021,47,43.0,4.0,250331.9,2728.362,94.31915
29105,78005,ACHERES,78260,ACHERES,NaN,48.971799,2.090039,5.0,NaN,Achères,...,Île-de-France,78005,26779,2021,223,95.0,128.0,276685.6,3734.265,75.94170
29106,78006,ADAINVILLE,78113,ADAINVILLE,NaN,48.720154,1.662325,6.0,NaN,Adainville,...,Île-de-France,78006,26780,2021,15,15.0,0.0,455405.1,3685.400,128.66670
29107,78007,AIGREMONT,78240,AIGREMONT,NaN,48.900838,2.015363,7.0,NaN,Aigremont,...,Île-de-France,78007,26781,2021,21,20.0,1.0,658779.9,4926.190,141.47620
29108,78009,ALLAINVILLE,78660,ALLAINVILLE AUX BOIS,NaN,48.467805,1.902487,9.0,NaN,Allainville,...,Île-de-France,78009,26782,2021,3,3.0,0.0,287900.0,1664.333,171.33330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29376,78677,VILLETTE,78930,VILLETTE,NaN,48.926279,1.693742,677.0,NaN,Villette,...,Île-de-France,78677,27029,2021,7,7.0,0.0,290364.3,2766.143,110.28570
29377,78681,VILLIERS LE MAHIEU,78770,VILLIERS LE MAHIEU,NaN,48.853337,1.773800,681.0,NaN,Villiers-le-Mahieu,...,Île-de-France,78681,27030,2021,5,5.0,0.0,261352.0,2897.800,92.20000
29378,78683,VILLIERS ST FREDERIC,78640,VILLIERS ST FREDERIC,NaN,48.820178,1.882508,683.0,NaN,Villiers-Saint-Frédéric,...,Île-de-France,78683,27031,2021,46,44.0,2.0,460481.1,3932.761,126.10870
29379,78686,VIROFLAY,78220,VIROFLAY,NaN,48.802404,2.171513,686.0,NaN,Viroflay,...,Île-de-France,78686,27032,2021,287,78.0,209.0,479045.4,6520.544,71.24739


In [ ]:
def q1(x):
    return x.quantile(0.25)

def q3(x):
    return x.quantile(0.75)

In [ ]:
mean_m2_departement = pd.pivot_table(data=prix, index=["nom_departement"], values="Prixm2Moyen", aggfunc=["mean", q1, q3])
mean_m2_departement

,mean,q1,q3
,Prixm2Moyen,Prixm2Moyen,Prixm2Moyen
nom_departement,,,
Ain,2235.777587,1662.166667,2653.858156
Aisne,1289.592416,986.500000,1540.666667
Allier,1125.098839,886.875000,1298.196429
Alpes-Maritimes,3366.643302,2009.954545,4508.419449
Alpes-de-Haute-Provence,2106.623975,1713.250000,2410.000000
...,...,...,...
Vendée,1820.764662,1401.333500,2073.174500
Vienne,1230.351865,1001.500000,1442.333250


In [ ]:
mean_m2_departement.columns = mean_m2_departement.columns.droplevel(0)

In [ ]:
mean_m2_departement.columns = ["mean_prixm2", "q1_prixm2", "q3_prixm2"]

In [ ]:
mean_m2_departement = mean_m2_departement.reset_index()
mean_m2_departement

,nom_departement,mean_prixm2,q1_prixm2,q3_prixm2
0,Ain,2235.777587,1662.166667,2653.858156
1,Aisne,1289.592416,986.500000,1540.666667
2,Allier,1125.098839,886.875000,1298.196429
3,Alpes-Maritimes,3366.643302,2009.954545,4508.419449
4,Alpes-de-Haute-Provence,2106.623975,1713.250000,2410.000000
...,...,...,...,...
89,Vendée,1820.764662,1401.333500,2073.174500
90,Vienne,1230.351865,1001.500000,1442.333250
91,Vosges,1143.733715,821.000000,1394.769000
92,Yonne,1279.941598,1031.484000,1456.083250


In [ ]:
columns = mean_m2_departement.columns

martinique = ["Martinique",2738.56,  1954, 3132]
guadeloupe = ["Guadeloupe", 3283.41, 2685, 3922]
reunion = ["La Réunion", 2771.48, 2154, 3238]
guyane = ["Guyane", 2433.11, 2139, 2574]

df_martinique = pd.DataFrame([martinique], columns=columns)
df_guadeloupe = pd.DataFrame([guadeloupe], columns=columns)
df_reunion = pd.DataFrame([reunion], columns=columns)
df_guyane = pd.DataFrame([guyane], columns=columns)

In [ ]:
full_m2 = pd.concat([mean_m2_departement, df_martinique, df_guadeloupe, df_reunion, df_guyane], axis=0).reset_index(drop=True)
full_m2


,nom_departement,mean_prixm2,q1_prixm2,q3_prixm2
0,Ain,2235.777587,1662.166667,2653.858156
1,Aisne,1289.592416,986.500000,1540.666667
2,Allier,1125.098839,886.875000,1298.196429
3,Alpes-Maritimes,3366.643302,2009.954545,4508.419449
4,Alpes-de-Haute-Provence,2106.623975,1713.250000,2410.000000
...,...,...,...,...
93,Yvelines,3516.851191,2808.901000,4008.104000
94,Martinique,2738.560000,1954.000000,3132.000000
95,Guadeloupe,3283.410000,2685.000000,3922.000000
96,La Réunion,2771.480000,2154.000000,3238.000000


In [ ]:
full_m2.to_csv('m2.csv', index=False)

In [ ]:
sum_mut_departement = pd.pivot_table(data=prix, index=["nom_departement"], values="Nb_mutations", aggfunc=["sum"])
sum_mut_departement

,sum
,Nb_mutations
nom_departement,
Ain,12389
Aisne,8306
Allier,5781
Alpes-Maritimes,89481
Alpes-de-Haute-Provence,3949
...,...
Vendée,18136
Vienne,8370


In [ ]:
sum_mut_departement.columns = sum_mut_departement.columns.droplevel(0)

In [ ]:
sum_mut_departement = sum_mut_departement.rename(columns={'': 'nom_departement', 'Nb_mutations': 'Total_Mutations'})
sum_mut_departement = sum_mut_departement.reset_index()
sum_mut_departement

,nom_departement,Total_Mutations
0,Ain,12389
1,Aisne,8306
2,Allier,5781
3,Alpes-Maritimes,89481
4,Alpes-de-Haute-Provence,3949
...,...,...
89,Vendée,18136
90,Vienne,8370
91,Vosges,5894
92,Yonne,9505


In [ ]:
columns = sum_mut_departement.columns

martinique = ["Martinique",3385]
guadeloupe = ["Guadeloupe", 3902]
reunion = ["La Réunion", 14128]
guyane = ["Guyane", 1376]

df_martinique = pd.DataFrame([martinique], columns=columns)
df_guadeloupe = pd.DataFrame([guadeloupe], columns=columns)
df_reunion = pd.DataFrame([reunion], columns=columns)
df_guyane = pd.DataFrame([guyane], columns=columns)

In [ ]:
full_nbre = pd.concat([sum_mut_departement, df_martinique, df_guadeloupe, df_reunion, df_guyane], axis=0).reset_index(drop=True)
full_nbre

,nom_departement,Total_Mutations
0,Ain,12389
1,Aisne,8306
2,Allier,5781
3,Alpes-Maritimes,89481
4,Alpes-de-Haute-Provence,3949
...,...,...
93,Yvelines,24785
94,Martinique,3385
95,Guadeloupe,3902
96,La Réunion,14128


In [ ]:
full_nbre.to_csv('nbre_ventes.csv', index=False)